# RoboSaw Motor Specification Requirements Calculations

## Usage Notes
1. If using stepper motor without feedback, real-world torque must not exceed 40% to 50% of maximum pull-out torque ([trinamic](https://www.trinamic.com/technology/motor-control-technology/dcstep/))
2. Generally the stepper motor should not exceed 600 rpm if driven without feedback ([omc-stepperonline](https://www.omc-stepperonline.com/support-stepper-motor/what-is-the-maximum-speed-highest-frequency-of-the-motor))

## Intake Stepper Motor Requirements

In [2]:
import numpy as np
import pandas as pd
import pint
import matplotlib.pyplot as plt
from IPython.display import display

In [3]:
# initial variables
ureg = pint.UnitRegistry()

cf_static_wood = 0.6
wt_wood = {'2_4': 1.3, '2_6' : 2.0, '4_4' : 4.8} # lb/ft
wt_wood = {key: val * ureg.lbf / ureg.ft for (key, val) in wt_wood.items()}
intake_wheel_diam = 2.25 * ureg.inch # in
max_length = 8 * ureg.ft # ft
fos = 1 # factor of safety

In [4]:
def force_frict(coef, wt):
    return coef * wt

def torque_frict(diam, force):
    return diam / 2 * force

In [5]:
force_max = force_frict(cf_static_wood, max_length * wt_wood[max(wt_wood, key=wt_wood.get)])
torque_max = torque_frict(intake_wheel_diam, force_max)

print(f"The maximum force is {force_max:.1f}")
print(f"The maximum torque with an intake wheel with diameter of {intake_wheel_diam} is {torque_max:.1f}\n")

print(f"Look for stepper motor with holding torque of {torque_max * fos:.1f}, {torque_max.to('ozf*inch') * fos:.1f}")

The maximum force is 23.0 force_pound
The maximum torque with an intake wheel with diameter of 2.25 inch is 25.9 force_pound * inch

Look for stepper motor with holding torque of 25.9 force_pound * inch, 414.7 force_ounce * inch


In [6]:
# must remove units for pandas compatibility

length = np.arange(1, max_length.magnitude, 1) * ureg.ft # ft

intake_recs = pd.DataFrame({'length' : length.magnitude})
for key, val in wt_wood.items():
    intake_recs["force_" + key] = force_frict(cf_static_wood, val.magnitude * intake_recs.length)
    intake_recs["torque_" + key] = torque_frict(intake_wheel_diam.magnitude, intake_recs["force_" + key])
display(intake_recs)

,length,force_2_4,torque_2_4,force_2_6,torque_2_6,force_4_4,torque_4_4
0,1,0.78,0.8775,1.2,1.35,2.88,3.24
1,2,1.56,1.7550,2.4,2.70,5.76,6.48
2,3,2.34,2.6325,3.6,4.05,8.64,9.72
3,4,3.12,3.5100,4.8,5.40,11.52,12.96
4,5,3.90,4.3875,6.0,6.75,14.40,16.20
5,6,4.68,5.2650,7.2,8.10,17.28,19.44
6,7,5.46,6.1425,8.4,9.45,20.16,22.68


## Turn Table Stepper Motor Requirements

In [7]:
def torque_pinion(force, pd_pinion):
    return force * pd_pinion / 2

def speed_table(vel_motor, gear_ratio, pd_pinion, diam_table):
    return (vel_motor / gear_ratio * pd_pinion / 2) / diam_table

In [8]:
pd_pinion = 1.5 * ureg.inch # in
force_rack = 8 * ureg.lbf # lbf
vel_motor = 600 * ureg.rpm # rpm
gear_ratio_stepper = 5
diam_table = 265 * ureg.millimeter # mm

torque_pinion_motor = torque_pinion(force_rack, pd_pinion)
print(f"The maximum torque required to move the rotary table is {torque_pinion_motor.to('lbf * in') * fos:.1f}, {torque_pinion_motor.to('ozf * in') * fos:.1f}")
max_speed_table = speed_table(vel_motor, gear_ratio_stepper, pd_pinion, diam_table)
print(f"The maximum speed of the turn_table is {max_speed_table.to('deg/s'):.1f}")

The maximum torque required to move the rotary table is 6.0 force_pound * inch, 96.0 force_ounce * inch
The maximum speed of the turn_table is 51.8 degree / second


## Saw Blade Linear Actuator Motor Requirements